<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C2/blob/master/W3/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Models: Auto-Complete
In this assignment, you will build an auto-complete system. Auto-complete system is something you may see every day

When you google something, you often have suggestions to help you complete your search.
When you are writing an email, you get suggestions telling you possible endings to your sentence.
By the end of this assignment, you will develop a prototype of such a system.

A key building block for an auto-complete system is a language model. A language model assigns the probability to a sequence of words, in a way that more "likely" sequences receive higher scores. For example,

* "I have a pen" is expected to have a higher probability than "I am a pen" since the first one seems to be a more natural sentence in the real world.

You can take advantage of this probability calculation to develop an auto-complete system.
Suppose the user typed

* "I eat scrambled" Then you can find a word x such that "I eat scrambled x" receives the highest probability. If x = "eggs", the sentence would be "I eat scrambled eggs"

While a variety of language models have been developed, this assignment uses N-grams, a simple but powerful method for language modeling.

* N-grams are also used in machine translation and speech recognition.
Here are the steps of this assignment:

1. Load and preprocess data
2. Load and tokenize data.
3. Split the sentences into train and test sets.
4. Replace words with a low frequency by an unknown marker <unk>.
5. Develop N-gram based language models
  * Compute the count of n-grams from a given data set.
  * Estimate the conditional probability of a next word with k-smoothing.
6. Evaluate the N-gram models by computing the perplexity score.
7. Use your own model to suggest an upcoming word given your sentence.

In [3]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt

--2020-10-20 09:40:06--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt’

en_US.twitter.txt   100%[===================>]   3.19M  --.-KB/s    in 0.1s    

2020-10-20 09:40:06 (25.2 MB/s) - ‘en_US.twitter.txt’ saved [3341555/3341555]



In [4]:
with open("en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------
